In [2]:
import numpy as np
import pandas as pd
from tools import seq_to_num, acc_score

In [3]:
df_train = pd.read_csv('../data/train.csv', index_col=0)
X_train, y_train = seq_to_num(df_train.Sequence, pad=False)

Using TensorFlow backend.


710

In [4]:
value_counts = {}
for seq in X_train:
    for val, count in zip(*np.unique(seq, return_counts=True)):
        if val not in value_counts:
            value_counts[val] = count
        else:
            value_counts[val] += count

In [5]:
sorted_counts = sorted(value_counts.items(), key=lambda x: x[1], reverse=True)

In [27]:
len(sorted_counts)

416016

In [14]:
sum([count for val, count in sorted_counts if val < 1000])

2422707

In [113]:
X_under = X_train.map(lambda seq: np.all([0 <= x < 1000 for x in seq]))
y_under = y_train.map(lambda val: 0 <= val < 1000)

In [121]:
X_under1000 = X_train[X_under & y_under]
y_under1000 = np.expand_dims(y_train[X_under & y_under], -1)

In [122]:
X_under1000.shape, y_under1000.shape

((24532,), (24532, 1))

In [28]:
# pad sequences
from keras.preprocessing.sequence import pad_sequences

In [36]:
np.median(X_under1000.map(lambda seq: len(seq)).unique())

80.0

In [128]:
Xp_under1000 = pad_sequences(X_under1000, maxlen=80, dtype='int32')
Xp_under1000 = np.expand_dims(Xp_under1000, 2)

In [129]:
Xp_under1000.shape

(24532, 80, 1)

In [104]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dropout, TimeDistributed, Dense, Flatten
def build_graph(seqlen):
    model = Sequential()
#     model.add(Embedding(seqlen, 300, mask_zero=True))
#     model.add(GRU(1000, return_sequences=True))
#     model.add(Dropout(0.5))
#     model.add(GRU(1000, return_sequences=True))
#     model.add(Dropout(0.5))
    model.add(GRU(128, input_shape=(seqlen, 1)))
    model.add(Dense(1000, activation='softmax'))
    return model

In [105]:
m = build_graph(80)

In [106]:
m.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [110]:
np.expand_dims(Xp_under1000, 2).shape

(25348, 80, 1)

In [126]:
hist = m.fit(Xp_under1000, y_under1000, 32, epochs=1)

Epoch 1/1
24532/24532 [==============================] - 36s 1ms/step - loss: 5.0332 - acc: 0.0820


In [131]:
pred = m.predict(Xp_under1000)
pred = np.argmax(pred, axis=1)

In [138]:
pred = np.argmax(pred, axis=1)

In [139]:
acc_score(y_under1000, pred)

0.09167617805315506